## 타이타닉 데이터 EDA
- 의사결정트리
- 선형회귀

### 타이타닉 로드

In [1]:
titanic_df = pd.read_csv("datas/titanic.csv")
titanic_df.tail(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


### 정보 확인

In [2]:
titanic_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### 1. 불필요한 feature를 제거

In [3]:
titanic_df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
# 정수, 실수 데이터 타입의 컬럼만 남김
# int형과, float형을 나눠서 concat으로 합친다.
columns1 = titanic_df.columns[titanic_df.dtypes == "int64"]
columns2 = titanic_df.columns[titanic_df.dtypes == "float64"]

result = pd.concat([titanic_df[columns1], titanic_df[columns2]], axis = 1)
result.tail(2)

,PassengerId,Survived,Pclass,SibSp,Parch,Age,Fare
889,890,1,1,0,0,26.0,30.00
890,891,0,3,0,0,32.0,7.75


In [5]:
# 원하는 컬럼만 뽑음
filtered_df_1 = titanic_df[["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]
filtered_df_1.tail(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
889,1,1,male,26.0,0,0,30.00,C
890,0,3,male,32.0,0,0,7.75,Q


In [6]:
filtered_df_1.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### 2. NaN 데이터 제거

#### 1. Age에서 없는 데이터(null)들을 빼준다.

In [7]:
filtered_df_2 = filtered_df_1[filtered_df_1["Age"].notnull()]
len(filtered_df_2)

714

In [8]:
# null 데이터가 아닌것의 갯수를 확인
len(filtered_df_1[filtered_df_1["Embarked"].notnull()])

889

In [9]:
len(filtered_df_1)

891

In [10]:
# notnull을 해주면 매트릭스형태로 된다. - filtered_df_1.notnull()
filtered_df = filtered_df_1[filtered_df_1.notnull().all(axis = 1)] # row로 변경하기 위해서 axis=1로 설정
len(filtered_df) # 891개의 데이터에서 712개로 줄어들은것을 확인

712

In [11]:
filtered_df.reset_index(drop = True, inplace = True)

### 3. One Hot Encoding 수행
- Sex, Embarked 컬럼을 원핫인코딩 수행

In [12]:
# 깊은 복사
one_hot_df_1 = filtered_df.copy()

In [13]:
# Sex 컬럼에서 male, female을 
# Male, Female 컬럼에 0, 1로 변경한다.
# apply함수를 사용해서 코딩
one_hot_df_1["Male"] = one_hot_df_1["Sex"].apply(lambda x : 1 if x == "male" else 0)
one_hot_df_1["Female"] = one_hot_df_1["Sex"].apply(lambda x : 1 if x == "female" else 0)

# Sex 컬럼을 삭제
one_hot_df_1.drop(columns = ["Sex"], inplace = True)

one_hot_df_1.tail()

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,Male,Female
707,0,3,39.0,0,5,29.125,Q,0,1
708,0,2,27.0,0,0,13.000,S,1,0
709,1,1,19.0,0,0,30.000,S,0,1
710,1,1,26.0,0,0,30.000,C,1,0
711,0,3,32.0,0,0,7.750,Q,1,0


In [14]:
# 깊은 복사
one_hot_df_2 = filtered_df.copy()

In [15]:
## pandas의 onehot 인코딩 함수 사용해서 원핫인코딩작성 - pd.get_dummies
# Sex 컬럼
one_hot_df_3 = pd.get_dummies(one_hot_df_2["Sex"])
one_hot_df_3.tail(3)

,female,male
709,1,0
710,0,1
711,0,1


In [16]:
## pandas의 onehot 인코딩 함수 사용해서 원핫인코딩작성 - pd.get_dummies
# Embarked 컬럼
one_hot_df_4 = pd.get_dummies(one_hot_df_2["Embarked"])
one_hot_df_4.tail(3)

,C,Q,S
709,0,0,1
710,1,0,0
711,0,1,0


In [17]:
# onehot으로 나온 데이터와 기존 데이터를 concat으로 합친다.
one_hot_df = pd.concat([one_hot_df_2, one_hot_df_3, one_hot_df_4], axis = 1)

# 기존 Sex, Embarked컬럼을 drop으로 삭제
one_hot_df.drop(columns = ["Sex", "Embarked"], inplace = True)

one_hot_df.tail(2)

,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S
710,1,1,26.0,0,0,30.00,0,1,1,0,0
711,0,3,32.0,0,0,7.75,0,1,0,1,0


### 4. 연령대와 Adult 컬럼을 만들기

In [18]:
# 깊은 복사
result_df = one_hot_df.copy()
result_df.tail(2)

,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S
710,1,1,26.0,0,0,30.00,0,1,1,0,0
711,0,3,32.0,0,0,7.75,0,1,0,1,0


In [19]:
# Ages 컬럼은 연령대 컬럼을 추가
result_df["Ages"] = ((result_df["Age"] // 10) * 10).astype("int") 
result_df.tail(2)

,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S,Ages
710,1,1,26.0,0,0,30.00,0,1,1,0,0,20
711,0,3,32.0,0,0,7.75,0,1,0,1,0,30


In [20]:
# Adult 컬럼은 20이상이면 1, 20미만이면 0으로 컬럼을 추가
result_df["Adult"] = 0
result_df.loc[result_df["Ages"] >= 20, "Adult"] = 1
result_df.tail(3)

,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S,Ages,Adult
709,1,1,19.0,0,0,30.00,1,0,0,0,1,10,0
710,1,1,26.0,0,0,30.00,0,1,1,0,0,20,1
711,0,3,32.0,0,0,7.75,0,1,0,1,0,30,1


### 5. 생존 모델 만들기
- 원래 데이터
- 탐색 후 전처리가 끝난 데이터
- 선형회귀분석, 디시전트리 알고리즘으로 풀기

In [21]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model # 선형 회귀 : 수치를 예측
from sklearn.tree import DecisionTreeClassifier # 의사결정트리 분류 모델
from sklearn.metrics import accuracy_score

In [22]:
filtered_df.tail(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
710,1,1,male,26.0,0,0,30.00,C
711,0,3,male,32.0,0,0,7.75,Q


In [23]:
one_hot_df.tail(2)

,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S
710,1,1,26.0,0,0,30.00,0,1,1,0,0
711,0,3,32.0,0,0,7.75,0,1,0,1,0


### filtered_df로 예측

In [37]:
# 독립변수, 종속변수
df_x = filtered_df[["Pclass", "Age", "SibSp", "Parch", "Fare"]] # 수치형 데이터만 # 독립변수
df_y = filtered_df[["Survived"]] # 예측하고자 하는 데이터 # 타겟변수(종속)

In [38]:
# train, test 분류
train_x, test_x, train_y, test_y = train_test_split(
    df_x, df_y, test_size = 0.1, random_state = 1
)

### 선형회귀 : 수치분석 예측할 때 사용

In [42]:
# 모델 생성
model = linear_model.LinearRegression() # 객체 생성
model.fit(train_x, train_y) # 학습 # 수치형만 있어야 가능하다 => 문자형이 있으면 에러난다.(선형회귀이기 때문에)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [44]:
# 예측
pred_y = model.predict(test_x)
pred_y = np.around(pred_y.flatten()).astype("int") # flatten() : 차원을 하나 축소 - 2차원에서 1차원으로 축소된것을 확인
pred_y

array([1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 1])

In [41]:
# 기존 데이터(test_y)와 예측된 데이터(pred_y)를 비교
round(accuracy_score(test_y, pred_y) * 100, 2) # 소수점으로 나와서 100을 곱했다.

62.5

### 의사결정 트리

In [34]:
model = DecisionTreeClassifier(max_depth = 2) # 객체 생성 
                              # max_depth : if문의 개체 수(컬럼의 갯수로 하는 것이 좋다.) - 많다고 좋은것이 아니다.
model.fit(train_x, train_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [35]:
# 예측
pred_y = model.predict(test_x)
pred_y = np.around(pred_y.flatten()).astype("int") # flatten() : 차원을 하나 축소 - 2차원에서 1차원으로 축소된것을 확인
pred_y

array([1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 1])

In [36]:
# 기존 데이터(test_y)와 예측된 데이터(pred_y)를 비교
round(accuracy_score(test_y, pred_y) * 100, 2) # 소수점으로 나와서 100을 곱했다.

73.61

### one_hot_df로 예측

In [81]:
# 독립변수, 종속변수
df_x = one_hot_df[one_hot_df.columns[1:]] # 수치형 데이터만 # 독립변수
df_y = one_hot_df[["Survived"]] # 예측하고자 하는 데이터 # 타겟변수(종속)

In [83]:
# train, test 분류
train_x, test_x, train_y, test_y = train_test_split(
    df_x, df_y, test_size = 0.1, random_state = 1
)

### 선형회귀 : 수치분석 예측할 때 사용

In [84]:
# 모델 생성
model = linear_model.LinearRegression() # 객체 생성
model.fit(train_x, train_y) # 학습 # 수치형만 있어야 가능하다 => 문자형이 있으면 에러난다.(선형회귀이기 때문에)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [85]:
# 예측
pred_y = model.predict(test_x)
pred_y = np.around(pred_y.flatten()).astype("int") # flatten() : 차원을 하나 축소 - 2차원에서 1차원으로 축소된것을 확인
pred_y

array([1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 1])

In [86]:
# 기존 데이터(test_y)와 예측된 데이터(pred_y)를 비교
round(accuracy_score(test_y, pred_y) * 100, 2) # 소수점으로 나와서 100을 곱했다.

76.39

### 의사결정 트리

In [103]:
model = DecisionTreeClassifier(max_depth = 9) # 객체 생성 
                              # max_depth : if문의 개체 수(컬럼의 갯수로 하는 것이 좋다.) - 많다고 좋은것이 아니다.
model.fit(train_x, train_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=9,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [104]:
# 예측
pred_y = model.predict(test_x)
pred_y = np.around(pred_y.flatten()).astype("int") # flatten() : 차원을 하나 축소 - 2차원에서 1차원으로 축소된것을 확인
pred_y

array([1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 1])

In [105]:
# 기존 데이터(test_y)와 예측된 데이터(pred_y)를 비교
round(accuracy_score(test_y, pred_y) * 100, 2) # 소수점으로 나와서 100을 곱했다.

84.72